In [1]:
import spglib
import random
import re

def cif2pos(path):  
    import ase.io
    import ase.io.vasp
    path = path
    tmp = ase.io.read(path)
    ase.io.vasp.write_vasp(str(path)+"2POSCAR",tmp)
    return str(path)+"2POSCAR"
def get_local(ftype="cif",path="./1.cif"):
    """
    if filetype not be ".cif"
    should trans be as POSCAR
    """
    pos = []
    if ftype=="cif":
        poscar_path = cif2pos(path)
        path = poscar_path
        
    elif ftype == "vasp":
        pass
    
    with open (path) as f:
            #pos = []
        flag = 0
        for i in f:
            if "Cartesian" in str(i):
                flag += 1
            if flag ==1:
                pos.append(i)
    pos = pos[1:]
    Local_pos= []
    for i in pos:
        local_pos = []
        x1 = i.split()[0] 
        x2 = i.split()[1]
        x3 = i.split()[2]
        local_pos.append(float(x1))
        local_pos.append(float(x2))
        local_pos.append(float(x3))
        Local_pos.append(local_pos)
    return len(Local_pos),Local_pos,poscar_path
def get_abc(path,ftype="cif"):
    I = []
    path = path
    print(path)
    if ftype == "cif":
        poscar_path = cif2pos(path)
        path = poscar_path
        print(poscar_path)
        I = []
        with open (path) as f:
            mat = f.readlines()[2:5]
            for i in mat:
                i = i.replace("\n","").split()
                T = []
                for t in i:
                    t = float(t)
                    T.append(t)
                I.append(T)  
    elif ftype == "vasp":
        #path = path
        I = []
        with open (path=path) as f:
            mat = f.readlines()[2:5]
            for i in mat:
                i = i.replace("\n","").split()
                T = []
                for t in i:
                    t = float(t)
                    T.append(t)
                I.append(T)       
    #print(I)
    return I

def refine_mat(positions,numbers,a_b_c):
    spg = []
    lat = []
    for _ in range(100):
        positions = positions
        numbers = numbers       # 与type有关
        try:
            #cell_mat = [] 
            #for x in range(10):
            #    cell_mat.append(random.uniform(0,a_b_c))
            #lattice = [[cell_mat[0], cell_mat[1], cell_mat[2]],
            #           [cell_mat[3], cell_mat[4], cell_mat[5]],
            #           [cell_mat[6], cell_mat[7], cell_mat[8]]]
            lattice = a_b_c
            cell = (lattice, positions, numbers)
            symmetry_refine = spglib.refine_cell(cell, symprec=1e-1)
            #print(symmetry_refine)
            symmetry = spglib.get_spacegroup(symmetry_refine, symprec=1e-1)
            spgnumber = str(symmetry)
            print(spgnumber)
            kk = re.compile(r'\d+')
            result = re.findall(kk,spgnumber)[-1]
            spg.append(result)
            lat.append(symmetry_refine)
            print(result)
            print("-----------------------------------------")
        except:
            #print("False")
            #print("-----------------------------------------")
            pass
    return spg,lat

def get_new_lat(path = "./1.cif"):
    """
    for RWNN
    only cif go on...
    """
    path = path
    elem_total,positions,poscar_path  = get_local( path = path)
    numbers = [1 for x in range(elem_total) ]
    a_b_c  = get_abc(path=path)
    cell = (a_b_c, positions, numbers)
    symmetry_refine = spglib.refine_cell(cell, symprec=1e-1)
    print(symmetry_refine)
    print(spglib.get_spacegroup(symmetry_refine, symprec=1e-1))
    envelope = symmetry_refine[0]
    return envelope,poscar_path

def get_poscar(path):
    """
    only format cif file
    """
    lat,poscar_path = get_new_lat(path)
    f = open(poscar_path, 'r')
    list_1 = f.readlines()
    list_1[2] =str(list(lat[0])).replace("[","").replace("]","").replace(",","  ")+"\n"
    list_1[3] =str(list(lat[1])).replace("[","").replace("]","").replace(",","  ")+"\n"
    list_1[4] =str(list(lat[2])).replace("[","").replace("]","").replace(",","  ")+"\n"
    f.close()
    f=open("refine_POSCAR","w")
    f.writelines(list_1)
    f.close()
def super_cell(path="POSCAR"):
    import ase.io.vasp
    ase.io.vasp.write_vasp("POSCAR_ex", ase.io.vasp.read_vasp(path)*(2,2,2), label='Supercell',direct=True,sort=True)

In [2]:
my_dir = "./"
get_poscar(get_new_lat(my_dir+"1.cif")[1])
super_cell(my_dir+"/refine_POSCAR")

./1.cif
./1.cif2POSCAR
(array([[ 3.661141  ,  0.        ,  0.        ],
       [-0.94460617,  3.98907777,  0.        ],
       [-0.71576518, -1.52412408,  4.38718928]]), array([[0.20971904, 0.34049349, 0.46219039],
       [0.66349509, 0.5801773 , 0.88865541],
       [0.0234434 , 0.55875149, 0.33504039],
       [0.78197289, 0.50406017, 0.39325839],
       [0.50960881, 0.91228279, 0.20403729],
       [0.67465939, 0.92795042, 0.07571801],
       [0.59794049, 0.36632386, 0.68814819],
       [0.08051557, 0.11025548, 0.94421364]]), array([1, 1, 1, 1, 1, 1, 1, 1], dtype=int32))
P1 (1)
./1.cif2POSCAR
./1.cif2POSCAR2POSCAR
(array([[ 3.661141  ,  0.        ,  0.        ],
       [-0.94460617,  3.98907777,  0.        ],
       [-0.71576518, -1.52412408,  4.38718928]]), array([[0.20971904, 0.34049349, 0.46219039],
       [0.66349509, 0.5801773 , 0.88865541],
       [0.0234434 , 0.55875149, 0.33504039],
       [0.78197289, 0.50406017, 0.39325839],
       [0.50960881, 0.91228279, 0.20403729],
      

/share/home/sutianhao/anaconda3/envs/pyxtal/lib/python3.7/site-packages/ase/io/cif.py:404: UserWarning: crystal system 'triclinic' is not interpreted for space group Spacegroup(1, setting=1). This may result in wrong setting!
  setting_name, spacegroup))


In [ ]:
#lat,poscar_path = get_new_lat("./1223.cif")

# def get_poscar(path):
#     """
#     only format cif file
#     """
#     lat,poscar_path = get_new_lat(path)
#     f = open(poscar_path, 'r')
#     list_1 = f.readlines()
#     list_1[2] =str(list(lat[0])).replace("[","").replace("]","").replace(",","  ")+"\n"
#     list_1[3] =str(list(lat[1])).replace("[","").replace("]","").replace(",","  ")+"\n"
#     list_1[4] =str(list(lat[2])).replace("[","").replace("]","").replace(",","  ")+"\n"
#     f.close()
#     f=open("POSCAR","w")
#     f.writelines(list_1)
#     f.close()

#get_poscar("./1223.cif")

#super_cell()

# def super_cell(path="POSCAR"):
#     import ase.io.vasp
#     ase.io.vasp.write_vasp("POSCAR_ex", ase.io.vasp.read_vasp("POSCAR")*(2,2,2), label='Supercell',direct=True,sort=True)

# for i in lat:
#     print(i)

# cat 1223.cif2POSCAR

# #a,b = refine_mat(positions=positions,numbers=numbers,a_b_c=a_b_c)



# def get_abc(path,ftype="cif"):
#     I = []
#     path = path
#     print(path)
#     if ftype == "cif":
#         poscar_path = cif2pos(path)
#         path = poscar_path
#         print(poscar_path)
#         I = []
#         with open (path) as f:
#             mat = f.readlines()[2:5]
#             for i in mat:
#                 i = i.replace("\n","").split()
#                 T = []
#                 for t in i:
#                     t = float(t)
#                     T.append(t)
#                 I.append(T)  
#     elif ftype == "vasp":
#         #path = path
#         I = []
#         with open (path=path) as f:
#             mat = f.readlines()[2:5]
#             for i in mat:
#                 i = i.replace("\n","").split()
#                 T = []
#                 for t in i:
#                     t = float(t)
#                     T.append(t)
#                 I.append(T)       
#     #print(I)
#     return I
    

# #a,b=get_local()